In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/hh/MLops/dataScienceProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_trainer(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=43)
        logger.info(f"train set shape {train_x.shape}")
        logger.info(f"test set shape {test_x.shape}")
        logger.info(f"train set shape {train_y.shape}")
        logger.info(f"test set shape {test_y.shape}")
        
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [11]:
try:
    obj = ConfigurationManager()
    model_trainer_config = obj.get_model_trainer()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-08-10 14:22:37,286: INFO: common: yaml file config/config.yaml is loaded successfully]
[2025-08-10 14:22:37,287: INFO: common: yaml file params.yaml is loaded successfully]
[2025-08-10 14:22:37,289: INFO: common: yaml file schema.yaml is loaded successfully]
[2025-08-10 14:22:37,289: INFO: common: created directory at artifacts]
[2025-08-10 14:22:37,290: INFO: common: created directory at artifacts/model_trainer]
[2025-08-10 14:22:37,293: INFO: 3367629389: train set shape (688, 15)]
[2025-08-10 14:22:37,293: INFO: 3367629389: test set shape (230, 15)]
[2025-08-10 14:22:37,293: INFO: 3367629389: train set shape (688, 1)]
[2025-08-10 14:22:37,293: INFO: 3367629389: test set shape (230, 1)]
